In [1]:
import os
import pandas as pd
import datetime as dt
import numpy as np
import re

In [2]:
today = pd.Timestamp(dt.datetime.today().date())
today_ymd = today.strftime('%Y%m%d')
today_y_m_d = today.strftime('%Y_%m_%d')

yesterday = today - dt.timedelta(days=1)
yesterday_ymd = yesterday.strftime('%Y%m%d')
yesterday_y_m_d = yesterday.strftime('%Y-%m-%d')

tomorrow = today + dt.timedelta(days=1)
tomorrow_ymd = tomorrow.strftime('%Y%m%d')
tomorrow_y_m_d = tomorrow.strftime('%Y-%m-%d')

print(today_y_m_d)

2022_09_21


In [3]:
column_names = {'Emiasid': 'emiasid', 'Emiasid ': 'emiasid' , 'emiasid ': 'emiasid', 'EMIAS_ID': 'emiasid', 'МО': 'mo', 
                'МУ': 'mu', 'Логин оператора': 'operator_login', 'Дата и время звонка': 'call_date', 
                'Результат звонка': 'call_result', 'Результат звонка ': 'call_result', 'Разговор состоялся': 'talk_result', 
                'Комменатрий': 'comment', 'Дата и время переноса': 'next_call_date', 'Контактный телефон': 'phone_number', 
                'ФИО врача': 'doctor_fio', 'ФИО помощника врача': 'assistant_fio', 'Логин помощника врача': 'operator_login',
                'Зал №': 'hall_number', 'Логин naumen': 'operator_login', 'МО закрепления': 'mo', 
                'Отсутствует на рабочем месте': 'absence', 'login': 'operator_login', 'procedure_rec_status': 'group_procedure',
                'patient_group': 'group_name'}

In [4]:
# путь папки с данными по звонкам
calls_data_path = r'\Users\murad_satabaev\Documents\GitHub\final_framework'
calls_data_path = calls_data_path.split('\\')
calls_data_path = os.path.join('C:\\', r'\\', *calls_data_path)
calls_data_path = os.path.join(calls_data_path, 'calls_data_20220802_today.xlsx')

In [5]:
# путь файла с текущими статусами пациентов
patients_current_statuses_path = r'Users\murad_satabaev\Documents\GitHub\final_framework\current_statuses_path'
patients_current_statuses_path = patients_current_statuses_path.split('\\')
patients_current_statuses_path = os.path.join('C:\\', r'\\', *patients_current_statuses_path)
print(patients_current_statuses_path)

C:\\Users\murad_satabaev\Documents\GitHub\final_framework\current_statuses_path


In [6]:
def find_phone_number(text):
    """Функция ищет номера телефонов в строке, фильтрует дефектные и возвращает один с приоритетом на мобильный"""
    import re
    defect_pattern = re.compile(r"""7[^94]\d{9}|0{7,}|1{7,}|2{7,}|3{7,}|4{7,}|5{7,}|6{7,}|7{7,}|8{7,}|9{7,}""")
    text = str(text)
    text = ['7' + re.sub(r'\D', '', x)[-10:] for x in re.split(r'[,;]', text)]
    text = list(filter(lambda x: len(x) == 11, text))
    text = list(filter(lambda x: False if re.search(defect_pattern, x) else True, text))
    text = sorted(text, reverse=True)
    text = ','.join(text)
    text = re.search(r'(9\d{9}|4\d{9})', text)
    if text:
        text = text.group()
    return text

In [7]:
%%time
calls_data_reading = pd.read_excel(calls_data_path)
calls_data_reading.shape[0]

CPU times: total: 1min 51s
Wall time: 1min 51s


334188

In [8]:
calls_data_reading = calls_data_reading.rename(columns=column_names)

In [9]:
%%time
calls_data_reading['emiasid'] = calls_data_reading['emiasid'].astype(str).str.replace('\.0', '')
calls_data_reading['call_date'] = calls_data_reading['call_date'].apply(lambda x: pd.to_datetime(x, dayfirst=True,
                                                                                                 errors='coerce').date())
calls_data_reading['next_call_date'] = \
calls_data_reading['next_call_date'].apply(lambda x: pd.to_datetime(x, dayfirst=True, errors='coerce').date())

<timed exec>:1: FutureWarning: The default value of regex will change from True to False in a future version.


CPU times: total: 1min 43s
Wall time: 1min 43s


In [10]:
calls_data_reading.call_date.unique()[-1]

datetime.date(2022, 9, 19)

In [11]:
august_for_september = pd.read_excel('all_august_to_september_emiasid.xlsx')
august_for_september['emiasid'] = august_for_september['emiasid'].astype(str).str.replace('\.0', '')

C:\Users\murad_satabaev\AppData\Local\Temp\ipykernel_1076\3882302525.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  august_for_september['emiasid'] = august_for_september['emiasid'].astype(str).str.replace('\.0', '')


In [12]:
monthes = {'august', 'september'}

In [13]:
%%time
for month in monthes:
    
    
    if 'september' in month:
        calls_data = calls_data_reading[calls_data_reading.call_date >= dt.date(2022, 9, 2)].reset_index(drop=True).copy()
    else:
        calls_data = calls_data_reading.copy()
        
    
    emiasid_touches = \
    calls_data.groupby('emiasid').agg({'call_date': 'count'}).rename(columns={'call_date': 'calls_amount'}).reset_index()
    
    last_calls_data = \
    calls_data.sort_values(by='call_date').drop_duplicates(subset=['emiasid'], keep='last').reset_index(drop=True)
    
    last_calls_data = last_calls_data[['emiasid', 'call_date', 'next_call_date', 'call_result', 'talk_result']]
    
    last_calls_data = last_calls_data.rename(
        columns={'call_date': 'last_call_date', 'call_result': 'last_call_result',
                'talk_result': 'last_talk_result'})
    
    last_comments = \
    calls_data[~calls_data.call_result.isin({'Нет ответа', 'Не дозвонились', 'Занято', 'Сброс'})][
        ['emiasid', 'comment', 'call_date']].sort_values(by='call_date') \
    .drop_duplicates(subset=['emiasid'], keep='last').reset_index(drop=True)[['emiasid', 'comment']] \
    .rename(columns={'comment': 'old_comment'})
    
    
    
    last_calls_data['emiasid'] = last_calls_data['emiasid'].astype(str).str.replace('\.0', '')
    last_calls_data['last_call_date'] = last_calls_data['last_call_date'].apply(lambda x: pd.to_datetime(x, dayfirst=True).date())
    last_calls_data['next_call_date'] = \
    last_calls_data['next_call_date'].apply(lambda x: pd.to_datetime(x, dayfirst=True).date())
    
    
    patients_current_statuses_reading = pd.read_excel(
        os.path.join(patients_current_statuses_path, rf'{month}_patients_current_statuses_{today_y_m_d}.xlsx'),
        usecols=['emiasid', 'status', 'doctor_fio', 'mo']
     )
    
    
    patients_current_statuses = patients_current_statuses_reading.rename(columns=column_names).copy()
    
    patients_current_statuses['emiasid'] = patients_current_statuses['emiasid'].astype(str).str.replace('\.0', '')
#     patients_current_statuses['phone_number'] = patients_current_statuses['phone_number'] \
#     .apply(lambda x: str(x).replace('.0', '') if not pd.isna(x) else x)
    
    
    patients_current_statuses_merged = patients_current_statuses.merge(emiasid_touches, on=['emiasid'], how='left')
    patients_current_statuses_merged.calls_amount = patients_current_statuses_merged.calls_amount.fillna(0)
    patients_current_statuses_merged = patients_current_statuses_merged.merge(last_calls_data, on=['emiasid'], how='left')
    patients_current_statuses_merged = patients_current_statuses_merged.merge(last_comments, on=['emiasid'], how='left')
    
#     if 'september' in month:
#         september_patients_current_statuses_merged = patients_current_statuses_merged.copy()
#     elif 'august' in month:
#         august_patients_current_statuses_merged = patients_current_statuses_merged.copy()
    
    
    if 'august' in month:
        patients_current_statuses_merged = patients_current_statuses_merged[patients_current_statuses_merged.emiasid.isin(
        august_for_september.emiasid)]
    
    patients_current_statuses_merged['month'] = month
    
    patients_current_statuses_merged = patients_current_statuses_merged[[
        'emiasid', 'status', 'old_comment', 'next_call_date', 
        'mo', 'doctor_fio', 
        'calls_amount', 'last_call_date',  'last_call_result', 'last_talk_result',  
        'month'        
    ]]
    

    
    patients_current_statuses_merged.to_excel(
        os.path.join('C:\\', 'Users', 'murad_satabaev', 'Documents', 'GitHub',
        'final_framework', 'merged_statuses', 
                     f'{month}_patients_current_statuses_with_calls_info_{today_y_m_d}.xlsx'), index=False)

<timed exec>:30: FutureWarning: The default value of regex will change from True to False in a future version.
<timed exec>:44: FutureWarning: The default value of regex will change from True to False in a future version.
<timed exec>:30: FutureWarning: The default value of regex will change from True to False in a future version.
<timed exec>:44: FutureWarning: The default value of regex will change from True to False in a future version.


CPU times: total: 1min 13s
Wall time: 1min 13s
